On commence par importer les librairies qui seront nécessaires. gym est la librairie de OpenAI qui contient les environnements de benchmark pour les algos de RL.

In [2]:
import numpy as np
import gym
from math import radians

Pour créer un environnement, on fait appel à gym.make("NOM_ENVIRONNEMENT").

Voici quelques exemples de commandes avec l'environnement CartPole-v1, où les états sont continus et les actions discrètes.

In [3]:
environment = gym.make("CartPole-v1")

# On extrait l'état de départ. Pour CartPole, le départ est stochastique: 
# le poteau et le chariot ne sont pas toujours aux mêmes points de départ.
s0 = environment.reset()

# L'environnement contient un attribut observation_space qui qualifie
# l'espace d'états. 
# Pour les états continus comme ici, on a accès à la proprité shape pour 
# connaître la forme des observations.
observation_dim = environment.observation_space.shape

# L'environnement contient aussi un attribut action_space qui qualifie
# l'espace d'actions.
# Pour les espaces à actions discrètes comme CartPole, celui-ci contient un 
# attribut n indiquant le nombre d'actions et les actions sont tout
# simplement les entiers de [n].
n_actions = environment.action_space.n # Only discrete action spaces

# On fait un pas dans l'environnement avec la fonction step(action).
# Cette fonction retourne (s, r, done), qui représentent respectivement
# le prochain état, la récompense et le fait que la trajectoire est finie ou pas.
s, r, done, _ = environment.step(1) # Actions are int

# Infos de base sur l'environnement
print(f"L'environnement {environment.spec.id} a des observations de la forme {observation_dim} et contient {n_actions} actions.")

L'environnement CartPole-v1 a des observations de la forme (4,) et contient 2 actions.


Pour la reproductibilité, on veut être capables d'initiliaser notre germe de nombres aléatoire à une valeur fixée.

Voici une fonction qui le fait pour vous:

In [4]:
def set_random_seed(environment, seed):
    environment.seed(seed)
    environment.action_space.seed(seed)
    environment.observation_space.seed(seed)
    np.random.seed(seed)

set_random_seed(environment, seed=42)

Pour gérer de manière tabulaire un espace d'états continu comme celui de CartPole, on procède à une discrétisation de l'espace.

La première étape d'une discrétisation est de déterminer les limites de l'espace d'état.

Celles-ci sont accessibles par "environment.observation_space.high" et "environment.observation_space.low".

Dans notre cas, l'état représente \[position chariot, vitesse chariot, angle poteau, vitesse angulaire poteau\].

Comme les vitesses ont des bornes infinies qui ne prètent pas bien à la discrétisation, on leur fixe les valeurs limites de 0.5 unité / t et 50 radians / t pour la vitesse du chariot et du poteau, respectivement.

In [5]:
upper_bounds = environment.observation_space.high
upper_bounds[1] = 0.5
upper_bounds[3] = radians(50)

lower_bounds = -upper_bounds

Pour la discrétisation, nous vous suggérons maintenant de discrétiser chaque composante en \[1, 1, 6, 12\] zones respectivement.

Votre discrétisation devrait retourner un tuple de taille 4 où chaque indice correspond à l'indice de la zone discrète représentant la composante continue. 

Chaque zone est de taille identique : (upper_i - lower_i) / N_i, où upper_i et lower_i sont les bornes supérieure et inférieure pour la composante i et N_i est le nombre de zones qui séparent la composante i.

Assurez-vous de gérer les cas limites où l'état en entrée contient des composantes en dehors de vos bornes (ramenez les à la borne la plus près).

In [6]:
def discretize(state, n_buckets=[1, 1, 6, 12]):
    """
    Takes as input a (4,) state of the CartPole environment and returns a (4,) tuple
    of discrete indexes for each component.

    The number of components is set by the n_buckets parameter and the split is 
    made according to the lower_bounds and upper_bounds global values.
    """
    
    #S'assurer que tout est dans les bornes.
    
    state = np.maximum(state, lower_bounds)
    state = np.minimum(state, upper_bounds)
    
    
    #Normalisation entre 0, 1 l'état à chaque index
    x = (state-lower_bounds)/(upper_bounds-lower_bounds)
    
    #transformation en un nombre entier entre 0 et n-1.
    
    x = x*(np.array(n_buckets)-1)
    
    discrete_x = np.rint(x).astype(int) # le astype pour qu'il puisse être vue comme un entier (un index) plus tard
    
    return tuple(discrete_x)

On bâtit maintenant un tableau qui va contenir nos approximations des Q valeurs selon la mise à jour Monte-Carlo.

Ce tableau sera de taille \[1, 1, 6, 16, 2\] pour correspondre à la taille de notre discrétisation et au nombre d'actions disponibles.

Implémentez la mise à jour des valeurs selon la formule de la moyenne mobile décrit à la section 2.4 du livre de Sutton et Barto.

In [7]:
class Table:
    def __init__(self, discrete_obs_shape, n_actions):
        """
        Initializes two [discrete_obs_shape, n_actions] ndarrays of zeros.
        One is to store the current q estimated values and the other is to store 
        the number of visits
        """
        #dimension du teableau est un tuple former de la conténation de discrete_obs_shape et n_actions
        #Typiquement, ici la shape est (1,1,6,16,2).
        
        self.table_shape= discrete_obs_shape + (n_actions,)
        
        #Tableaux de 0 pour initialiser les valeurs de Q
        self.Q=np.zeros(self.table_shape, dtype=np.float32)
        self.N_visits=np.zeros(self.table_shape,dtype=np.uint)
        
    def store_return(self, obs_index, action, g):
        """
        Stores return g for (obs_index, action) pair.
        Updates the corresponding q value according to recurrent running
        mean formula.
        Also updates number of visits to said pair.
        """
        #g est le gain cumulé
        
        #Obention de l'index de la table Q à utiliser.
        Q_idx=obs_index + (action,)
        
        #Q value précédente associée à la paire (état-action)
        prev_Q_value=self.Q[Q_idx]
        
        #Nombre de visites qu'on avait pour cet élément là.
        prev_N_visites = self.N_visits[Q_idx]
        
        #Mise à jour de la valeur de Q pour la paire (état-action) 
        #à l'aide la formule donnée par la section 2.4 du livre de Sutton et Barto.
        
        #self.Q[Q_idx]=prev_Q_value+(1/(prev_N_visites+1))*(g-prev_Q_value)
        
        #Méthode d'Audrey
        self.Q[Q_idx]=(prev_Q_value*prev_N_visites+g)/(prev_N_visites+1)
        
        #Mise à journ du nombre de visite pour la paire état-action
        self.N_visits[Q_idx] += 1


On implémente aussi une politique epsilon-greedy. 

Notez qu'on peut piger une action aléatoire de l'environnement avec "action_space.sample()".

In [8]:
def choose_action(state_idx, table, epsilon, action_space):
    """
    Takes as input a state_idx, a Table table, an epsilon and an action_space parameters.

    table has a q attribute that can be read with table.q[state_idx] to get the estimated
    q values of all actions at this specific state index.
    """
    if np.random.random() < epsilon:
        return action_space.sample()
    else:
        return np.argmax(table.Q[state_idx])

Implémentez maintenant l'algorithme MC Control First-visit de la section 5.4 du livre de Sutton.

Utilisez les fonctions discretize(), choose_action() et l'objet Table que vous venez de créer.

Faites changer epsilon en le faisant commencer à 1.0 et en le multipliant par 0.99 à la fin de chaque épisode. Conservez un epsilon minimal de 0.1.

Générez 5000 trajectoires avec ces paramètres. Votre agent devrait converger à une récompense près de 500.

La convergence est très sensible et c'est donc normal de voir votre agent obtenir des trajectoires de 500 points de récompenses suivies de trajectoires de 20 points de récompense.

Retournez la table finale après votre entraînement, elle sera utile plus tard.

In [10]:
#l'algorithme MC Control First-visit de la section 5.4 du livre de Sutton.
def on_policy_mc(gamma=1.0):
    environment = gym.make("CartPole-v1")
    set_random_seed(environment, seed=23)

    # On crée le Table avec n_buckets en tuple
    table = Table((1, 1, 6, 12), environment.action_space.n)
    
    #Les actions possibles
    actions=list(range(environment.action_space.n))
    
    epsilon = 1.0
    
    #Une boucle sur 5000 trajectoires
    for i in range(5000):
        #On stock les états, les actions et les rewards rencontrés dans notre trajectoire.
        #La trajectoire sera donc une liste de tuple
        trajectory = []
        
        #Flag pour savoir si la trajectoire est terminée
        trajectory_done = False 
    
        #Mise à jour de epsilon
        epsilon=max(0.99*epsilon, 0.1)
        
        #On commence dans un état chosit
        
        s=environment.reset()
        
        while not trajectory_done:
            #On discretise notre état (obtention des indexs associés
            s_idx = discretize(s)
            
            #On sélectionne une action
            a=choose_action(s_idx,table,epsilon,environment.action_space)
            
            #obtention du prochain état next_s, de la récompense r, et du flag, en jouant l'action choisie.
            next_s, r, trajectory_done, _ = environment.step(a)
            
            #mise à jour de la trajectoire (on était dans quel état, on fait quel action et on a obtenu quel reward)
            
            trajectory.append((s_idx, a, r))
            
            #mise à jour de l'état en cours
            s=next_s
        
        #Analyse de la trajectoire pour faire notre update
        
        G=0
        # G sera la reward cumulée au temp t
        
        #On se garde une trace des états qui ont été visités (car on se mettra en mode firs-visit)
        visited=[]
        
        #On parcourt à l'inverse notre trajectoire pour que nos gain cumulée veule dire quelque chose.
        for state_idx, action, reward in reversed(trajectory):
            G = gamma * G +reward
            
            # si la paire (state_idx, action) n'est pas dans une paire (state_idx, action) visité, on met
            # à jour la valeur état action.
            if (state_idx, action) not in visited:
                table.store_return(state_idx, action ,G)
                
                visited.append((state_idx,action))

        if i%50 ==0:
            print('Après {} trajectoires, nous avons G_0 = {}'.format(i,G))
            
    return table

mc_table = on_policy_mc()

Après 0 trajectoires, nous avons G_0 = 20.0
Après 50 trajectoires, nous avons G_0 = 78.0
Après 100 trajectoires, nous avons G_0 = 77.0
Après 150 trajectoires, nous avons G_0 = 83.0
Après 200 trajectoires, nous avons G_0 = 93.0
Après 250 trajectoires, nous avons G_0 = 192.0
Après 300 trajectoires, nous avons G_0 = 34.0
Après 350 trajectoires, nous avons G_0 = 112.0
Après 400 trajectoires, nous avons G_0 = 119.0
Après 450 trajectoires, nous avons G_0 = 16.0
Après 500 trajectoires, nous avons G_0 = 154.0
Après 550 trajectoires, nous avons G_0 = 28.0
Après 600 trajectoires, nous avons G_0 = 96.0
Après 650 trajectoires, nous avons G_0 = 46.0
Après 700 trajectoires, nous avons G_0 = 255.0
Après 750 trajectoires, nous avons G_0 = 48.0
Après 800 trajectoires, nous avons G_0 = 98.0
Après 850 trajectoires, nous avons G_0 = 43.0
Après 900 trajectoires, nous avons G_0 = 37.0
Après 950 trajectoires, nous avons G_0 = 111.0
Après 1000 trajectoires, nous avons G_0 = 28.0
Après 1050 trajectoires, nous 

Implémentez maintenant un objet QTable qui contiendra les valeurs de Q mais mises à jour l'objectif de Q-learning.

Vous pouvez réutiliser le code de Table que vous avez fait plus haut.

In [9]:
#6.5 of Sutton and Barto's book.
class QTable:
    def __init__(self, discrete_obs_shape, n_actions):
        """
        Same as in Table except we do not need a ndarray for the number of visits anymore.
        """
        #dimension du teableau est un tuple former de la conténation de discrete_obs_shape et n_actions
        #Typiquement, ici la shape est (1,1,6,16,2).
        
        self.table_shape= discrete_obs_shape + (n_actions,)
        
        #Tableaux de 0 pour initialiser les valeurs de Q
        self.Q=np.zeros(self.table_shape, dtype=np.float32)
        
    def update(self, obs_index, action, reward, next_obs_index, learning_rate, gamma): 
        """
        Updates the Q estimation for state-action pair (obs_index, action).

        The update rule can be found in section 6.5 of Sutton and Barto's book.
        """
        
        prev_Q_value = self.Q[obs_index][action]
        increment = reward + gamma * self.Q[next_obs_index].max() - prev_Q_value #C'est l'erreur TD
        
        self.Q[obs_index][action] += learning_rate * increment
        

Implémentez maintenant l'algorithme Q-learning de la section 6.5 du livre de Sutton et Barto en utilisant maintenant la QTable que vous venez d'implémenter.

Utilisez le même régime pour epsilon que ce que vous avez utilisé pour le MC control.

Utilisez learning_rate = epsilon pour chaque mise à jour.

Vous devriez encore une fois être en mesure de réutiliser une bonne partie du code que vous avez fait pour le MC Control.

In [10]:
def q_learning(gamma=1.0, learning_rate=1.0):
    environment = gym.make("CartPole-v1")
    set_random_seed(environment, seed=42)

    # On change ici qu'on crée une QTable.
    table = QTable((1, 1, 6, 12), environment.action_space.n)
    
    #Les actions possibles
    actions=list(range(environment.action_space.n))
    
    epsilon=1.0
    
    for i in range(500):
        
        trajectory_done = False
        
        #On stoquera les rewards seulement pour pouvoir imprimer notre gain à la fin
    
        reward=[]
        
        s = environment.reset()
        
        s_idx = discretize(s)
        
        while not trajectory_done:
            a=choose_action(s_idx, table, epsilon, environment.action_space)
            
            next_s, r, trajectory_done, _ = environment.step(a)
            
            next_s_idx = discretize(next_s)
    
            table.update(s_idx, a, r, next_s_idx, learning_rate, gamma)
        
            s_idx=next_s_idx
            
            reward.append(r)
    
        epsilon=max(epsilon*0.99,0.1)
        learning_rate=max(learning_rate*0.99,0.1)
        
        if i % 10 == 0:
            print('Après {} trajectoires, nous avons G_0 = {}, esilon={}'.format(i,np.sum(reward),epsilon))
    
    return table

q_table = q_learning()

Après 0 trajectoires, nous avons G_0 = 11.0, esilon=0.99
Après 10 trajectoires, nous avons G_0 = 20.0, esilon=0.8953382542587163
Après 20 trajectoires, nous avons G_0 = 13.0, esilon=0.8097278682212583
Après 30 trajectoires, nous avons G_0 = 21.0, esilon=0.7323033696543974
Après 40 trajectoires, nous avons G_0 = 20.0, esilon=0.6622820409839835
Après 50 trajectoires, nous avons G_0 = 13.0, esilon=0.5989560064661611
Après 60 trajectoires, nous avons G_0 = 24.0, esilon=0.5416850759668536
Après 70 trajectoires, nous avons G_0 = 43.0, esilon=0.4898902730042049
Après 80 trajectoires, nous avons G_0 = 19.0, esilon=0.44304798162617254
Après 90 trajectoires, nous avons G_0 = 23.0, esilon=0.40068465295154065
Après 100 trajectoires, nous avons G_0 = 26.0, esilon=0.36237201786049694
Après 110 trajectoires, nous avons G_0 = 148.0, esilon=0.3277227574378037
Après 120 trajectoires, nous avons G_0 = 402.0, esilon=0.2963865873992079
Après 130 trajectoires, nous avons G_0 = 500.0, esilon=0.26804671691687

La librairie gym vient aussi avec des outils permettant de visualiser le comportement des agents dans l'environnement.

La fonction run() ci-dessous crée un wrapper autour de l'environnement et va déposer dans un dossier "demos" les statistiques de votre agent sur une trajectoire test sur l'environnement.

Cette fonction affiche et sauvegarde aussi une vidéo montrant le déroulement en temps réel de la trajectoire dans l'environnement.

In [11]:
def run(table, epsilon=0.1):
    environment = gym.make("CartPole-v1")
    set_random_seed(environment, seed=42)

    # On ajoute un wrapper Monitor et on écrit dans un folder demos les données et la vidéo
    env = gym.wrappers.Monitor(environment, 'demos', force=True)

    done = False
    s_idx = discretize(env.reset())
    while not done:
        # On rajoute un appel à render pour faire afficher les pas dans l'environnement
        env.render()
        action = choose_action(s_idx, table, epsilon, env.action_space)
        next_s, r, done, _ = environment.step(action)            
        next_s_idx = discretize(next_s)
        s_idx = next_s_idx

Exemple avec l'agent appris par Q-learning :

In [12]:
run(q_table)

DependencyNotInstalled: Found neither the ffmpeg nor avconv executables. On OS X, you can install ffmpeg via `brew install ffmpeg`. On most Ubuntu variants, `sudo apt-get install ffmpeg` should do it. On Ubuntu 14.04, however, you'll need to install avconv with `sudo apt-get install libav-tools`.